In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.executing_eagerly()

True

In [3]:
filter_height = 1
filter_width = 3
in_channels = 50
out_channels = 60

batch_size = 2
N = 8

In [7]:
np.random.seed(1234)

#x = np.random.randn(batch_size, 1, N, in_channels).astype(np.float32)
x = np.random.randn(batch_size, N, in_channels).astype(np.float32)
kernel_init = np.random.randn(filter_height, filter_width, in_channels, out_channels)

In [8]:
kernel = tf.get_variable(
    'kernel',
    [filter_height, filter_width, in_channels, out_channels],
    dtype=tf.float32,
    initializer=tf.constant_initializer(kernel_init))

In [9]:
# (batch_size, 1, N - filter_width + 1, out_channels)
tf.nn.conv2d(x, kernel, [1, 1, 1, 1], 'VALID')

InvalidArgumentError: input must be 4-dimensional[2,8,50] [Op:Conv2D]

In [29]:
# 一番最初のチャネルの先頭部分のstride
x[0][0][0:3][:, 0]

array([0.47143516, 0.8410088 , 0.29120535], dtype=float32)

In [50]:
# 一番最後のチャネルの先頭部分のstride
x[0][0][0:3][:, 49]

array([ 0.21526858, -0.5118813 ,  2.3657687 ], dtype=float32)

In [58]:
# 一番最初のoutput filterの一番最初のinput filter
kernel_init[:, :, :, 0][0, :, 0]

array([1.24990251, 0.92659274, 1.09751853])

In [47]:
# 一番最初のoutput filterの一番最後のinput filter
kernel_init[:, :, :, 0][0, :, 49]

array([0.29332215, 0.85244479, 0.05491674])

In [57]:
# 上記を要素単位で掛け合わせて合計をとったのが
# 出力の一番最初の値
np.sum(x[0][0][0:3] * kernel_init[:, :, :, 0][0, :])

19.806728275227247

In [65]:
# 同じfilterを一つずらして計算した結果が次の計算結果
# 出力では[0, 0, 1, 0]に格納されている
np.sum(x[0][0][1:4] * kernel_init[:, :, :, 0][0, :])

-9.943988432878225

In [67]:
# 頭のstrideと２つめのoutput filterとの畳込み
# 出力の[0, 0, 0, 1]に格納されている
np.sum(x[0][0][0:3] * kernel_init[:, :, :, 1][0, :])

7.162849036481002